# Initial Normalization of Images

In [31]:
from PIL import Image, ImageEnhance
import os
from tqdm import tqdm
import cv2

In [6]:
def normalize_images(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir,img))
    new_image = image.resize((64, 64))
    new_image.save(os.path.join(dataset_dir, new_dir, img))

Resize Landscape images to 64x64

In [10]:
for i in tqdm(os.listdir('dataset/landscapes')):
    normalize_images('dataset', 'landscapes', 'landscape_64x64', i)

100%|██████████| 4319/4319 [02:28<00:00, 28.99it/s]


In [37]:
def color_to_bw(dataset_dir, dir, new_dir, img):
    image = Image.open(os.path.join(dataset_dir, dir, img))
    image_new = image.convert('L')
    image_new.save(os.path.join(dataset_dir, new_dir, img))

In [38]:
for i in tqdm(os.listdir('dataset/landscape_64x64')):
    color_to_bw('dataset', 'landscape_64x64', 'landscape_64x64_bw', i)

100%|██████████| 4319/4319 [00:45<00:00, 94.53it/s] 


# Models

## Siggraph 2016

In [42]:
%%bash

FILENAME="colornet.t7"
FILEURL="http://iizuka.cs.tsukuba.ac.jp/data/colornet.t7"
FILEMD5="c88fa2bb6dc9f942a492a7dc7009b966"

echo "Downloading the colorization model (663M)..."
wget --continue -O "$FILENAME" -- "$FILEURL"

echo "Checking integrity (md5sum)..."
OS=`uname -s`
if [ "$OS" = "Darwin" ]; then
  CHECKSUM=`cat $FILENAME | md5`
  else
  CHECKSUM=`md5sum $FILENAME | awk '{ print $1 }'`
fi

if [ "$CHECKSUM" != "$FILEMD5" ]; then
  echo "Integrity check failed. File is corrupt!"
  echo "Try running this script again and if it fails remove '$FILENAME' before trying again."
  exit 1
fi 
echo -e "Download finished successfully! Time to \e[31mc\e[32mo\e[33m\e[34ml\e[35mo\e[36mr\e[37mi\e[91mz\e[92me\e[93m\e[94m!\e[m"


Couldn't find program: 'bash'


In [12]:
cd ..

d:\Documents\Escuela\8vo. Semestre\Deep Learning\Proyecto_Final


In [13]:
!git clone https://github.com/nagadomi/distro.git torch --recursive

Submodule path 'exe/env': checked out 'd28424ac5b638686410304c60c31693849f3d1dd'

Cloning into 'torch'...
Submodule 'exe/env' (https://github.com/torch/env.git) registered for path 'exe/env'
Submodule 'exe/luajit-rocks' (https://github.com/torch/luajit-rocks.git) registered for path 'exe/luajit-rocks'
Submodule 'exe/qtlua' (https://github.com/torch/qtlua.git) registered for path 'exe/qtlua'
Submodule 'exe/trepl' (https://github.com/torch/trepl.git) registered for path 'exe/trepl'
Submodule 'extra/argcheck' (https://github.com/torch/argcheck.git) registered for path 'extra/argcheck'
Submodule 'extra/cudnn' (https://github.com/nagadomi/cudnn.torch.git) registered for path 'extra/cudnn'
Submodule 'extra/cunn' (https://github.com/nagadomi/cunn.git) registered for path 'extra/cunn'
Submodule 'extra/cutorch' (https://github.com/nagadomi/cutorch.git) registered for path 'extra/cutorch'
Submodule 'extra/graph' (https://github.com/torch/graph) registered for path 'extra/graph'
Submodule 'extra/lua-cjson' (https://github.com/mpx/lua-cjson) registered for path 'extra/lua-cjson


Submodule path 'exe/luajit-rocks': checked out '411f4b9d9c4be176d4aab965ebfce50911583e14'
Submodule path 'exe/qtlua': checked out '8b80419ca2458e172b0d245999114f2b9d4ac118'
Submodule path 'exe/trepl': checked out 'e5e17e3a56997123bd6c66cb8575175d3a6945bb'
Submodule path 'extra/argcheck': checked out 'b3b32c060c4269c25d11cf9f983c291d9448e1ee'
Submodule path 'extra/cudnn': checked out 'fcc8c042f024621932ebffce67655402dd6c634c'
Submodule path 'extra/cunn': checked out '9f057781ab34d76370d73cd85f2de31dd0fc7de2'
Submodule path 'extra/cutorch': checked out '927e45666c2056f4bd583c8a7ba32cadf19c372a'
Submodule path 'extra/graph': checked out '37dac07a19eb3809b8081514bc781206241863a0'
Submodule path 'extra/lua-cjson': checked out 'e8972ac754788d3ef10a57a36016d6c3e85ba20d'
Submodule path 'extra/luaffifb': checked out '610ce4dc6de318c8f0eb027f052be081349097be'
Submodule path 'extra/luafilesystem': checked out '3c4e563d9c140319e28c419f2710b51a2f6d6d24'
Submodule path 'extra/moses': checked out 'd

In [10]:
!th siggraph_2016/siggraph2016_colorization/colorize.lua dataset/landscape_64x64_bw/00000000.jpg [./ouptut.jpg]

'th' is not recognized as an internal or external command,
operable program or batch file.
